In [ ]:
#import db

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2,application_1682627123102_0003,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
from pyspark.sql import Window
from pyspark.sql import functions as F 
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pyspark.pandas as ps
import pandas as pd

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#appliance db
df = spark.read.json('s3://cis4567/project1/Appliances.json')
df = df.drop('image', 'reviewTime', 'style', 'unixReviewTime', 'verified')
df = df.na.drop()
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------+--------------------+--------------+--------------------+--------------------+----+
|      asin|overall|          reviewText|    reviewerID|        reviewerName|             summary|vote|
+----------+-------+--------------------+--------------+--------------------+--------------------+----+
|1118461304|    5.0|Not one thing in ...|A3NHUQ33CFH3VM|              Greeny|Clear on what lea...|   2|
|1118461304|    5.0|As I began to rea...|A26JGAM6GZMM4V|       Robert Morris|How and why it is...|  10|
|1118461304|    4.0|A compelling book...|A3VF3A5A3O04E1|           spsanders|Strangely Interes...|   2|
|1118461304|    5.0|Alan Gregerman's ...|A14DW5UMQ1M96O|        Mike Jannini|The Power of Inno...|   3|
|1118461304|    5.0|I usually find bu...|A2I5QME4S7CLFI|            Laura F.|A New Twist on Ou...|   2|
|1118461304|    4.0|Seriously! We alw...|A33HIV8RXRDM88|     Kanishk Rastogi|First 20 pages wi...|   2|
|1118461304|    5.0|Very thought prov...| ALZ2S4839OZUQ|Darrell 

In [ ]:
#appliance meta db
df_meta = spark.read.json('s3://cis4567/project1/meta_Appliances.json')
df_meta = df_meta.select('asin', 'brand', 'category', 'title', 'price')
df_meta = df_meta.na.drop()
df_meta.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------------+--------------------+--------------------+--------------------+
|      asin|         brand|            category|               title|               price|
+----------+--------------+--------------------+--------------------+--------------------+
|7301113188|    Tupperware|[Appliances, Refr...|Tupperware Freeze...|                    |
|7861850250|    Tupperware|[Appliances, Refr...|2 X Tupperware Pu...|               $3.62|
|8792559360|The Cigar Book|[Appliances, Part...|The Cigar - Momen...|             $150.26|
|9792954481|     Caraselle|[Appliances, Part...|Caraselle 2X 50G ...|.a-box-inner{back...|
|B00002N5EL|  EATON Wiring|[Appliances, Part...|EATON Wiring 39CH...|               $3.43|
|B00002N7HY|       Leviton|[Appliances, Part...|Leviton 5050 B01-...|               $6.98|
|B00002N7IL|       Leviton|[Appliances, Part...|Leviton 5206 50 A...|                    |
|B00002N9OE|       Amerock|[Appliances, Part...|Amerock BP5322-BJ...|               $7.68|

In [ ]:
#join both db
df_join = df.join(df_meta, on = 'asin', how = 'inner')
df_join.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------+--------------------+--------------+--------------------+--------------------+----+----------------+--------------------+--------------------+------+
|      asin|overall|          reviewText|    reviewerID|        reviewerName|             summary|vote|           brand|            category|               title| price|
+----------+-------+--------------------+--------------+--------------------+--------------------+----+----------------+--------------------+--------------------+------+
|B001HSAP6S|    5.0|A great hard to f...|A1WUZXABDBTFK6|              jackal|Good replacement ...|   2|  American Metal|[Appliances, Part...| Aluminum Range H...|$14.50|
|B001KGX0XC|    5.0|If your oven didn...|A105G3GP8ARVMS|              g-star|Neat rack for Whi...|   4|       Whirlpool|[Appliances, Part...|Whirlpool Part Nu...|      |
|B0046C04XQ|    5.0|Something spilled...|A3UPW6K80CG9OG|           Happy Ten|Very pleased with...|   3|           Supco|[Appliances, Part...|Replaceme

In [ ]:
#preprocess

In [ ]:
#rename column
df_join = df_join.withColumnRenamed('overall', 'rating')\
    .withColumnRenamed('reviewText', 'review')\
    .withColumnRenamed('reviewerID', 'customer ID')\
    .withColumnRenamed('reviewerName', 'customer name')\
    .withColumnRenamed('vote', 'rating vote')\
    .withColumnRenamed('title', 'product name')

df_join.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------+--------------------+--------------+--------------------+--------------------+-----------+----------------+--------------------+--------------------+------+
|      asin|rating|              review|   customer ID|       customer name|             summary|rating vote|           brand|            category|        product name| price|
+----------+------+--------------------+--------------+--------------------+--------------------+-----------+----------------+--------------------+--------------------+------+
|B001HSAP6S|   5.0|A great hard to f...|A1WUZXABDBTFK6|              jackal|Good replacement ...|          2|  American Metal|[Appliances, Part...| Aluminum Range H...|$14.50|
|B001KGX0XC|   5.0|If your oven didn...|A105G3GP8ARVMS|              g-star|Neat rack for Whi...|          4|       Whirlpool|[Appliances, Part...|Whirlpool Part Nu...|      |
|B0046C04XQ|   5.0|Something spilled...|A3UPW6K80CG9OG|           Happy Ten|Very pleased with...|          3|           

In [ ]:
#check for dup
df_join.join(df_join.groupBy(df_join.columns).agg((F.count('*') > 1).cast('string').alias('Duplicate_indicator')),
             on = df_join.columns,how = 'inner').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------+--------------------+--------------+----------------+--------------------+-----------+-------------+--------------------+--------------------+------+-------------------+
|      asin|rating|              review|   customer ID|   customer name|             summary|rating vote|        brand|            category|        product name| price|Duplicate_indicator|
+----------+------+--------------------+--------------+----------------+--------------------+-----------+-------------+--------------------+--------------------+------+-------------------+
|B00004YWK2|   5.0|It worked great. ...|A3LZ13IS49DF09|       Hennes123|Worked great last...|          2|Dundas Jafine|[Appliances, Part...|Dundas Jafine CHK...| $9.90|               true|
|B00004YWK2|   5.0|It worked great. ...|A3LZ13IS49DF09|       Hennes123|Worked great last...|          2|Dundas Jafine|[Appliances, Part...|Dundas Jafine CHK...| $9.90|               true|
|B00006IV17|   1.0|Dear product dist...|A17THI32CXXUMG|

In [ ]:
#check and drop nan
df_join = df_join.na.drop()
df_join.where(col('review').isNull()).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+------+------+-----------+-------------+-------+-----------+-----+--------+------------+-----+
|asin|rating|review|customer ID|customer name|summary|rating vote|brand|category|product name|price|
+----+------+------+-----------+-------------+-------+-----------+-----+--------+------------+-----+
+----+------+------+-----------+-------------+-------+-----------+-----+--------+------------+-----+

In [ ]:
#only select review and rating column (and convert rating to int)
df_join_select = df_join.select('review', col('rating').cast('Int'))
df_join_select.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------+
|              review|rating|
+--------------------+------+
|A great hard to f...|     5|
|If your oven didn...|     5|
|Something spilled...|     5|
|I'm a housewife w...|     5|
|Worked like a cha...|     5|
|I looked all over...|     4|
|Deceptive produce...|     2|
|If you read some ...|     1|
|Fittings were che...|     1|
|This a very nice ...|     4|
|Disappointed I re...|     1|
|When you order th...|     1|
|When i got my del...|     1|
|Excellent quality...|     5|
|I received a chea...|     1|
|I have bought a c...|     5|
|Ordered it a mont...|     2|
|The only braided ...|     4|
|Make sure you get...|     5|
|These appear to b...|     4|
+--------------------+------+
only showing top 20 rows

In [ ]:
#train text according to 75-25 rule
split_data = df_join_select.randomSplit([0.75, 0.25])

train_data = split_data[0]
test_data = split_data[1]

print(train_data.count(), test_data.count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

49828 16720

In [ ]:
from nltk.tokenize import word_tokenize
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#tokenize text
tokenizer = Tokenizer(inputCol="review", outputCol="SentimentWords")
tokenizedTrain = tokenizer.transform(train_data)
tokenizedTrain.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------+--------------------+
|              review|rating|      SentimentWords|
+--------------------+------+--------------------+
|(this is a copy o...|     5|[(this, is, a, co...|
|--Edgestar Portab...|     4|[--edgestar, port...|
|1st fridge was de...|     2|[1st, fridge, was...|
|4 months after pu...|     1|[4, months, after...|
|6inch pipe wouldn...|     3|[6inch, pipe, wou...|
|<a data-hook="pro...|     5|[<a, data-hook="p...|
|<a data-hook="pro...|     5|[<a, data-hook="p...|
|<div id="video-bl...|     4|[<div, id="video-...|
|<div id="video-bl...|     5|[<div, id="video-...|
|<div id="video-bl...|     4|[<div, id="video-...|
|A complete piece ...|     1|[a, complete, pie...|
|A couple small it...|     4|[a, couple, small...|
|A good buy that i...|     5|[a, good, buy, th...|
|A good buy that i...|     5|[a, good, buy, th...|
|A hard to find it...|     3|[a, hard, to, fin...|
|A joke.  Save you...|     1|[a, joke., , save...|
|A joke.  Save you...|     1|[a

In [ ]:
#remove stopwords
swr = StopWordsRemover(inputCol=tokenizer.getOutputCol(), 
                       outputCol="MeaningfulWords")
SwRemovedTrain = swr.transform(tokenizedTrain)
SwRemovedTrain.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------+--------------------+--------------------+
|              review|rating|      SentimentWords|     MeaningfulWords|
+--------------------+------+--------------------+--------------------+
|(this is a copy o...|     5|[(this, is, a, co...|[(this, copy, rev...|
|--Edgestar Portab...|     4|[--edgestar, port...|[--edgestar, port...|
|1st fridge was de...|     2|[1st, fridge, was...|[1st, fridge, del...|
|4 months after pu...|     1|[4, months, after...|[4, months, purch...|
|6inch pipe wouldn...|     3|[6inch, pipe, wou...|[6inch, pipe, fit...|
|<a data-hook="pro...|     5|[<a, data-hook="p...|[<a, data-hook="p...|
|<a data-hook="pro...|     5|[<a, data-hook="p...|[<a, data-hook="p...|
|<div id="video-bl...|     4|[<div, id="video-...|[<div, id="video-...|
|<div id="video-bl...|     5|[<div, id="video-...|[<div, id="video-...|
|<div id="video-bl...|     4|[<div, id="video-...|[<div, id="video-...|
|A complete piece ...|     1|[a, complete, pie...|[complete, pie

In [ ]:
#hash words - use to prep the below code
hashTF = HashingTF(inputCol=swr.getOutputCol(), outputCol="features")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#hash words -- USE THIS FOR LOGISTIC REGRESSION
numericTrain = hashTF.transform(SwRemovedTrain)
numericTrain.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------+--------------------+--------------------+--------------------+
|              review|rating|      SentimentWords|     MeaningfulWords|            features|
+--------------------+------+--------------------+--------------------+--------------------+
|(this is a copy o...|     5|[(this, is, a, co...|[(this, copy, rev...|(262144,[3121,383...|
|--Edgestar Portab...|     4|[--edgestar, port...|[--edgestar, port...|(262144,[2437,352...|
|1st fridge was de...|     2|[1st, fridge, was...|[1st, fridge, del...|(262144,[3230,155...|
|4 months after pu...|     1|[4, months, after...|[4, months, purch...|(262144,[1546,898...|
|6inch pipe wouldn...|     3|[6inch, pipe, wou...|[6inch, pipe, fit...|(262144,[12650,25...|
|<a data-hook="pro...|     5|[<a, data-hook="p...|[<a, data-hook="p...|(262144,[908,1415...|
|<a data-hook="pro...|     5|[<a, data-hook="p...|[<a, data-hook="p...|(262144,[908,1415...|
|<div id="video-bl...|     4|[<div, id="video-...|[<div, id="video-...

In [ ]:
#hash words -- USE THIS FOR RANDOM FOREST
numericTrain2 = numericTrain.select('MeaningfulWords', col('rating').cast('Int'), 'features')
numericTrain2.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------+--------------------+
|     MeaningfulWords|rating|            features|
+--------------------+------+--------------------+
|[(this, copy, rev...|     5|(262144,[3121,383...|
|[--edgestar, port...|     4|(262144,[2437,352...|
|[1st, fridge, del...|     2|(262144,[3230,155...|
|[4, months, purch...|     1|(262144,[1546,898...|
|[6inch, pipe, fit...|     3|(262144,[12650,25...|
|[<a, data-hook="p...|     5|(262144,[908,1415...|
|[<a, data-hook="p...|     5|(262144,[908,1415...|
|[<div, id="video-...|     4|(262144,[3121,800...|
|[<div, id="video-...|     5|(262144,[4389,553...|
|[<div, id="video-...|     4|(262144,[2306,271...|
|[complete, piece,...|     1|(262144,[34343,44...|
|[couple, small, i...|     4|(262144,[21641,24...|
|[good, buy, is,, ...|     5|(262144,[3121,953...|
|[good, buy, is,, ...|     5|(262144,[3121,953...|
|[hard, find, item...|     3|(262144,[2437,181...|
|[joke., , save, m...|     1|(262144,[10617,14...|
|[joke., , save, m...|     1|(2

In [ ]:
#logistic regression

In [ ]:
from pyspark.ml.classification import LogisticRegression

In [ ]:
#logestic regression -- need to fine tune hyperparam as required
lr = LogisticRegression(labelCol="rating", featuresCol="features", 
                        maxIter=10, regParam=0.01)
model = lr.fit(numericTrain)
print ("Training Done")

In [ ]:
#prepare testing data
tokenizedTest = tokenizer.transform(test_data)
SwRemovedTest = swr.transform(tokenizedTest)
numericTest = hashTF.transform(SwRemovedTest)
numericTest.show()

In [ ]:
#prepare prediction
raw_prediction = model.transform(numericTest)

In [ ]:
#prediction
Final_prediction = raw_prediction.select("MeaningfulWords", "prediction", "rating")
Final_prediction.show()

In [ ]:
#evulation
otal_True=Final_prediction.filter(Final_prediction['prediction']==Final_prediction['rating']).count()
Alldata=Final_prediction.count()
Accuracy=Total_True/Alldata
print(Accuracy*100)

In [ ]:
#random forest

In [ ]:
from pyspark.ml.classification import RandomForestClassifier

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#random forest -- need to fine tune hyperparam as required
rf = RandomForestClassifier(labelCol="rating", \
                            featuresCol="features", \
                            numTrees = 100, \
                            maxDepth = 4, \
                            maxBins = 32)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
rfModel = rf.fit(numericTrain2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Interrupted by user


In [ ]:
tree_predictions = rfModel.transform(testData)

In [ ]:
tree_predictions.filter(tree_predictions['prediction'] == 0) \
    .select("clean_text","category","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)